In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
     

from sklearn.datasets import make_regression

X , y = make_regression(n_samples = 100 , n_features = 3 , n_informative = 3, n_targets = 1, noise = 1)
X_df = pd.DataFrame(X)
y_df = pd.DataFrame(y)
df = pd.concat([X_df , y_df] , axis = 1)
df

,0,1,2,0
0,-1.332427,-1.320302,-0.268061,-162.886845
1,0.201406,0.757609,-1.554441,-7.541373
2,0.066059,0.634654,-0.428337,34.705433
3,-1.085308,0.745927,-0.332219,14.761618
4,0.126204,1.861458,-0.410481,139.512982
...,...,...,...,...
95,0.233621,-1.440013,-0.205136,-126.271568
96,-1.336013,0.026421,1.282328,25.303024
97,-1.373593,-0.244382,0.504724,-36.542839
98,0.198548,-0.515298,-1.595562,-117.206844


In [10]:
x1 = [2104 , 1416 , 1532 , 852]
x2 = [5 , 3 , 3 , 2]
x3 = [1 , 2 , 2 , 1]
x4 = [45 , 40 , 30, 36]
y = [460, 232 , 315 , 178]
#Values of x1 , x2 , x3 , x4 , y
#Target is y price #

X = np.array([x1 , x2 , x3 , x4])
X = X.T
y = np.array(y)
y = y.T
X = pd.DataFrame(X)
y = pd.DataFrame(y)

df = pd.concat([X , y] , axis = 1)
print(df.head())

      0  1  2   3    0
0  2104  5  1  45  460
1  1416  3  2  40  232
2  1532  3  2  30  315
3   852  2  1  36  178


In [11]:
#To find the coefficients for mutiple linear regression#
#We need to set a bias column for X#
# The bias column is now added within the MultipleRegressor class for clarity and to avoid double-adding.
# If you intend to add it here, ensure the MultipleRegressor class does not add it again.
X

,0,1,2,3
0,2104,5,1,45
1,1416,3,2,40
2,1532,3,2,30
3,852,2,1,36


In [12]:
from sklearn.model_selection import train_test_split
# Extract features and target from df, ensuring X does not have a bias column at this stage
X_features = df.iloc[:, :4].values # Assuming first 4 columns of df are features
y_target = df.iloc[:, 4].values.reshape(-1, 1) # Last column of df is the target

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=42)
print(X_train.shape)
     

(3, 4)


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train , y_train)
ypred = model.predict(X_test)

ypred

array([[304.79089708]])

In [14]:
class MultipleRegressor():
  def __init__(self):
    self.coef_ = None
    self.intercept_ = None
  def fit(self , X_train , y_train):
    # Insert bias column at the beginning of X_train
    X_train = np.insert(X_train , 0 , 1 , axis = 1)

    # Calculate coefficients using the normal equation with pseudo-inverse for robustness
    # (X.T @ X)^-1 @ X.T @ y
    # Using np.linalg.pinv (pseudo-inverse) to handle potential singular matrices (when n_samples < n_features)
    X_transpose_X_inv = np.linalg.pinv(np.dot(X_train.T , X_train))
    X_2 = np.dot(X_transpose_X_inv , X_train.T)
    self.coef_ = np.dot(X_2 , y_train)

    # The first element of self.coef_ is the intercept, and the rest are feature coefficients
    self.intercept_ = self.coef_[0]
    self.coef_ = self.coef_[1:]

  def predict(self , X_test):
    # Predict y values based on X_test, feature coefficients, and intercept
    # X_test should not include the bias column here, as the intercept is added separately
    return np.dot(X_test , self.coef_) + self.intercept_

In [15]:
model = MultipleRegressor()
model.fit(X_train , y_train)
ypred = model.predict(X_test)
ypred

array([[291.01671678]])